In [ ]:
from fastai.text import *

In [ ]:
path = untar_data(URLs.IMDB_SAMPLE)
path

In [ ]:
df = pd.read_csv(path/'texts.csv', header=None)
df.head()

In [ ]:
data = TextLMDataBunch.from_csv(path, 'texts.csv', bptt=10, bs=8)
# data_clas = TextClasDataBunch.from_csv(path, 'texts.csv', vocab=data_lm.train_ds.vocab, bs=42)

In [ ]:
# from fastai_data import MusicTokenizer
# path = Path('data/midi/v6/midi_transcribe/shortdur/')
# data = TextLMDataBunch.load(path, cache_name='tmp/hook', bs=4, bptt=20)
# data.valid_ds.x.processor[0] = TokenizeProcessor(tokenizer=MusicTokenizer())
# t = data.train_ds[0][0]

In [6]:
learn = language_model_learner(data, TransformerXL)
learn.unfreeze()


/home/ubuntu/fastai/fastai/text/learner.py:198: UserWarning: There are no pretrained weights for that architecture yet!
  warn("There are no pretrained weights for that architecture yet!")


In [7]:
learn.predict('hello there', n_words=10)

'hello there courage ishtar setups sentimental singlehandedly 19th walking packed till gradually'

In [8]:
import pdb

In [9]:
def beam_search(self, text:str, n_words:int, top_k:int=10, beam_sz:int=1000, temperature:float=1., sep:str=' ',
                decoder=decode_spec_tokens):
    "Return the `n_words` that come after `text` using beam search."
    ds = self.data.single_dl.dataset
    self.model.reset()
    xb, yb = self.data.one_item(text)
    nodes = None
    xb = xb.repeat(top_k, 1)
    nodes = xb.clone()
    scores = xb.new_ones(1).float()
    with torch.no_grad():
        for k in progress_bar(range(n_words), leave=False):
            out = F.log_softmax(self.model(xb)[0][:,-1], dim=-1)
            values, indices = out.topk(top_k, dim=-1)
            scores = (-values * scores[:,None]).view(-1)
            indices_idx = torch.arange(0,nodes.size(0))[:,None].expand(nodes.size(0), top_k).contiguous().view(-1)
            sort_idx = scores.argsort()[:beam_sz]
            scores = scores[sort_idx]
            nodes = torch.cat([nodes[:,None].expand(nodes.size(0),top_k,nodes.size(1)),
                            indices[:,:,None].expand(nodes.size(0),top_k,1),], dim=2)
            nodes = nodes.view(-1, nodes.size(2))[sort_idx]
            self.model[0].select_hidden(indices_idx[sort_idx])
            xb = nodes[:,-1][:,None]
    if temperature != 1.: scores.div_(temperature)
    pdb.set_trace()
    node_idx = torch.multinomial(1-scores, 1).item()
    return text + sep + sep.join(decoder(self.data.vocab.textify([i.item() for i in nodes[node_idx]], sep=None)))


In [10]:
beam_search(learn, 'hello there', n_words=5)

> <ipython-input-9-377f4724ca8e>(26)beam_search()
-> node_idx = torch.multinomial((1-scores)/scores.sum(), 1).item()
(Pdb) continue


RuntimeError: CUDA error: device-side assert triggered

In [ ]:
learn.beam_search('hello there', n_words=10)

In [ ]:
%debug

In [20]:
# data.train_dl.sampler

# data.train_dl.dl.dataset

In [13]:
tdl = data.train_dl.new()

In [21]:
data.train_dl.sampler

In [22]:
data.train_dl.on_epoch_begin()

In [23]:
len(data.train_ds)

799

In [24]:
len(data.train_dl.dl.dataset.idx.idx)

799

In [28]:
len(data.train_dl)

3182

In [25]:
it = iter(data.train_dl)

In [26]:
n = next(it)
[data.vocab.textify(x) for x in n[0]]

["xxbos xxmaj first of all ; it 's very xxunk",
 'his life since there is no specific risk in it',
 '\n\n xxmaj another sore point is that the whole production',
 ': everyone stumbles through their lines like wooden dolls ,',
 'concept of this film , especially the " film makers',
 '" companion " , and there was no xxunk .',
 'who wished that he / she is older / bigger',
 "'re aware that this music is similar to xxmaj newman"]

In [27]:
n2 = next(it)
[data.vocab.textify(x) for x in n2[0]]

['to try describe way of history only from positions of',
 '( xxmaj burt xxmaj reynolds ) ; a passionate family',
 'seemed to be one long set of product xxunk ,',
 'especially this xxmaj indian girl , she may be pretty',
 'were then attacked " aspect that the case seems to',
 'xxmaj there was also no " xxmaj grace " (',
 '; that just is so xxunk \n\n xxmaj just watch',
 "'s other work and while listening to the soundtrack you"]

In [14]:
it2 = iter(tdl)

In [15]:
n = next(it2)
[data.vocab.textify(x) for x in n[0]]

["no matter what script he 's following . xxmaj his character , xxmaj hunt xxmaj stevenson , is a likable , affable xxunk , a natural leader with a wise - ass xxunk . xxmaj but he has a fatal flaw common to many of us : he does n't want to disappoint anyone . xxmaj he 'll distract the crowd with inspirational xxunk , and even lie , rather",
 'sex , then killed by human xxunk . xxmaj there is plenty of gore , including the classic " something is wrong with her , oh no , it \'s half the person she used to be " and the xxunk murder caused by panic . xxmaj but you can also find funny stuff like intestines pulled through someone \'s ass and a guy running in the woods then finding',
 'patience from the viewer . xxmaj it must be said that some of the humor falls flat and the usual romantic misunderstandings that occur in any xxmaj fred xxmaj astaire film of this period are given conventional treatment . xxmaj only the musical interludes give the story the lift it needs . \n\n xxmaj some pleas

In [70]:
n[0].shape

torch.Size([4, 20])